In [1]:
#http://sdsawtelle.github.io/blog/output/week9-recommender-andrew-ng-machine-learning-with-python.html?fbclid=IwAR3AHrpbxYCKhbGnzOyWHdm3IXppK7qjg73ULwAyQhR5e24_3693xyNqK90
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import math

%matplotlib inline

In [2]:
import os
print(os.getcwd())

/Users/YiHsien/Documents/kaggle/AnimeRecommendations


In [64]:
anime = pd.read_csv('anime.csv')
rating = pd.read_csv('rating.csv')

In [4]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [5]:
rating.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [9]:
mergedata = pd.merge(rating,anime,on=['anime_id','anime_id'])
mergedata = mergedata[mergedata['type']=='Movie']
mergedata.head(10)

,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
98741,1,442,-1,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Drama, Historical, Shounen,...",Movie,1,7.17,120571
98742,5,442,4,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Drama, Historical, Shounen,...",Movie,1,7.17,120571
98743,54,442,-1,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Drama, Historical, Shounen,...",Movie,1,7.17,120571
98744,99,442,2,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Drama, Historical, Shounen,...",Movie,1,7.17,120571
98745,128,442,7,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Drama, Historical, Shounen,...",Movie,1,7.17,120571
98746,129,442,8,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Drama, Historical, Shounen,...",Movie,1,7.17,120571
98747,171,442,10,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Drama, Historical, Shounen,...",Movie,1,7.17,120571
98748,188,442,-1,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Drama, Historical, Shounen,...",Movie,1,7.17,120571
98749,190,442,5,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Drama, Historical, Shounen,...",Movie,1,7.17,120571
98750,201,442,10,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Drama, Historical, Shounen,...",Movie,1,7.17,120571


In [79]:
## train and test

In [10]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(mergedata, test_size=0.3)

In [11]:
print(train.shape,test.shape)

(736398, 9) (315600, 9)


In [80]:
## 將電影類別換成數字

In [71]:
from sklearn.base import BaseEstimator, TransformerMixin
class CounterEncoder(BaseEstimator, TransformerMixin):
    """依照出現頻率進行編碼, 頻率由高到低的index = 0, 1, 2, 3 ..., 以此類推"""
    def fit(self, y):
        counter = pd.Series(y).value_counts()
        self.enc = dict(zip([None] + counter.index.tolist(), range(len(counter) + 1)))
        self.invEnc = dict(zip(self.enc.values(), self.enc.keys()))
        self.classes_ = counter.index.values
        return self

    def transform(self, y):
        return pd.Series(y).map(self.enc).fillna(0).values

    def fit_transform(self, y, **fit_params):
        return self.fit(y).transform(y)
    
    def inverse_transform(self, y):
        return pd.Series(y).map(self.invEnc).values


class OrderedMapper(CounterEncoder):
    def fit(self, y):
        uniq = pd.Series(y).unique()
        self.enc = dict(zip(uniq, range(len(uniq))))
        self.invEnc = dict(zip(range(len(uniq)), uniq))
        self.classes_ = uniq
        return self

In [67]:
from collections import defaultdict, Counter, OrderedDict
def doMovies(movies):
    """處理 movie: genres 轉換成數字"""
    movies = movies.reset_index(drop=True)
    movies.loc[movies.genre == "(no genres listed)", "genre"] = ""
    movies['genre']=movies['genre'].fillna('')
    movies["genre"] = movies.genre.str.split(",")
    genreMap = Counter()
    movies.genre.map(genreMap.update)
    om = OrderedMapper().fit([e[0] for e in genreMap.most_common()])
    movies["genre"] = movies.genre.map(lambda lst: [om.enc[e] for e in lst])
    return movies, om

In [74]:
movie_trans, genres_enc = doMovies(anime)

In [77]:
from sklearn.preprocessing import minmax_scale
movie_trans["members"] = minmax_scale(movie_trans.members.fillna(movie_trans.members.median()))
movie_trans.head()

/Users/YiHsien/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64.
  warnings.warn(msg, DataConversionWarning)


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"[13, 7, 9, 12]",Movie,1,9.37,0.197872
1,5114,Fullmetal Alchemist: Brotherhood,"[1, 15, 11, 3, 17, 27, 4]",TV,64,9.26,0.782770
2,28977,Gintama°,"[1, 6, 18, 30, 41, 2, 4]",TV,51,9.25,0.112689
3,9253,Steins;Gate,"[49, 47]",TV,24,9.17,0.664325
4,9969,Gintama&#039;,"[1, 6, 18, 30, 41, 2, 4]",TV,51,9.16,0.149186


In [78]:
## leave on out

In [108]:
#拿掉無評分的資料
train = train[train['rating_x']!=-1]
test = test[test['rating_x']!=-1]
print(train.shape,test.shape)

(589505, 9) (252648, 9)


In [103]:
#train1 = pd.merge(train[['user_id','anime_id','rating_x']],movie_trans[['anime_id','genre','rating','members']], how="left", on="anime_id")
#test1 = pd.merge(test[['user_id','anime_id','rating_x']],movie_trans[['anime_id','genre','rating','members']], how="left", on="anime_id")

In [95]:
def loo_preprocess(data, movie_trans, train_hist=None, is_train=True):
    """以leave one out方式產生 train data, test data"""
    queue = []
    data = pd.merge(data[['user_id','anime_id','rating_x']],movie_trans[['anime_id','genre','rating','members']], how="left", on="anime_id")
    columns=["user_id", "anime_id", "genre", "rating","members", "candidate_movie_id","rating_x"]
    for u, df in data.groupby("user_id"):
        df = df.sort_values("rating_x", ascending=False)
        if not is_train:
            user_movies_hist = train_hist.query("user_id == {}".format(u)).anime_id
        for i, (_, r) in enumerate(df.iterrows()):
            if is_train:
                queue.append([int(r.user_id),
                              df.anime_id[:i].tolist() + df.anime_id[i + 1:].tolist(),
                              r.genre, r.rating, r.members, int(r.anime_id), r.rating_x])
            else:
                # queue.append([int(r.userId), df.movieId[:i].tolist() + df.movieId[i + 1:].tolist(), r.genres, r.avg_rating, r.year, int(r.movieId), r.rating])
                # all_hist = set(user_movies_hist.tolist() + df.movieId[:i].tolist())
                all_hist = set(user_movies_hist.tolist())
                queue.append([int(r.user_id),
                              list(all_hist - set([int(r.anime_id)])),
                              r.genre, r.rating, r.members, int(r.anime_id), r.rating_x])
    return pd.DataFrame(queue, columns=columns)

In [104]:
trProcessed = loo_preprocess(train, movie_trans)
teProcessed = loo_preprocess(test, movie_trans)

In [105]:
trProcessed.head()

,user_id,anime_id,genre,rating,members,candidate_movie_id,rating_x
0,3,"[1943, 9917, 4026, 1526, 2201, 11737, 1119, 76...","[5, 11, 12]",8.93,0.459852,199,10
1,3,"[199, 9917, 4026, 1526, 2201, 11737, 1119, 769...","[23, 32, 26, 36, 2, 47]",8.15,0.161062,1943,8
2,3,"[199, 1943, 4026, 1526, 2201, 11737, 1119, 769...","[5, 6, 11, 3, 8]",6.94,0.016880,9917,8
3,3,"[199, 1943, 9917, 1526, 2201, 11737, 1119, 769...","[1, 15, 3, 8]",7.10,0.032201,4026,7
4,3,"[199, 1943, 9917, 4026, 2201, 11737, 1119, 769...","[1, 15, 11, 3, 8]",7.40,0.046239,1526,7


In [106]:
teProcessed.head()

,user_id,anime_id,genre,rating,members,candidate_movie_id,rating_x
0,3,"[1689, 1122, 16782, 1121, 2847, 1764, 12671]","[5, 3, 7]",7.81,0.075234,9760,9
1,3,"[9760, 1122, 16782, 1121, 2847, 1764, 12671]","[13, 7, 10]",8.10,0.319584,1689,8
2,3,"[9760, 1689, 16782, 1121, 2847, 1764, 12671]","[1, 15, 6, 3, 8, 2]",7.10,0.044578,1122,7
3,3,"[9760, 1689, 1122, 1121, 2847, 1764, 12671]","[13, 36, 7, 10]",8.35,0.187133,16782,7
4,3,"[9760, 1689, 1122, 16782, 2847, 1764, 12671]","[5, 6, 11, 3, 8]",6.93,0.042843,1121,7


In [107]:
print(train.shape,test.shape)

(589505, 9) (252648, 9)
